# A COMPARISON OF CONVOLUTIONAL NEURAL NETWORKS FOR GLOTTAL CLOSURE INSTANT DETECTION FROM RAW SPEECH

This is an example of a Python code to train and test an InceptionV3-1D model, a deep one-dimensional convolutional neural network (CNN), for detecting glottal closure instants (GCIs) in the speech signal. See the [corresponding paper](paper/matousek_ICASSP2021_paper.pdf) for more details.

[Keras](https://keras.io/) (v2.3.1) with [TensorFlow](https://www.tensorflow.org/) (v1.15.3) backend are used to train and evaluate the CNN model.

Prerequisities are stored in the [requirements](requirements.txt) file.

Firstly, we make import libraries.

In [1]:
import os
import os.path as osp
import numpy as np
import random as pyrandom
import tensorflow as tf
import sklearn.metrics as skm
from keras.models import model_from_json
import librosa as lr
import utils
import wav_manip as wm
import gci_utils as gu
from inception1D import InceptionV31D

Using TensorFlow backend.


## Data

To show the training and evaluation of the InceptionV3-1D model, we describe data firstly. Note that just a [sample of data](data/sample) will be used in this tutorial (40 waveforms for training and 2 waveforms for testing from 2 voice talents). In the [corresponding paper](paper/matousek_ICASSP2021_paper.pdf), 3200 waveforms from 16 voice talents were used.

The following sample of data is used:
* `spc8 ...` speech waveforms downsampled to 8 kHz
* `negpeaks ...` indeces of negative peaks in the (filtered) speech waveform
* `targets ...` ground truth GCIs associated with the negative peaks (1=GCI, 0=non-GCI)

We used the [Multi-Phase Algorithm](http://www.sciencedirect.com/science/article/pii/S0167639311000094) (MPA) to detect GCIs from the contemporaneous electroglottograph (EGG) signal and used the detected GCIs as the ground truth ones.

As can be seen, the number of GCIs and non-GCIs in our data is heavily unbalanced:

In [2]:
utt_list = np.loadtxt('data/sample/train.txt', 'str')
targets = np.hstack([np.load(osp.join('data/sample/targets', u+'.npy')) for u in utt_list])

print('# peaks:   ', len(targets))
print('# GCIs:    ', len(targets[targets > 0]))
print('# non-GCIs:', len(targets[targets == 0]))

# peaks:    10990
# GCIs:     8659
# non-GCIs: 2331


This is caused by the 8kHz sampling as there are fewer peaks in unvoiced segments taken as non-GCIs.

## Training and evaluating the CNN model

The following code sets the randomness and tries to ensure reproducibility

In [3]:
seed_value = 7
# Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED'] = str(seed_value)
os.environ['CUDA_VISIBLE_DEVICES'] = ''
# Set python built-in pseudo-random generator at a fixed value
pyrandom.seed(seed_value)
# Set numpy pseudo-random generator at a fixed value
np.random.seed(seed_value)
# Set the tensorflow pseudo-random generator at a fixed value
tf.set_random_seed(seed_value)

# Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Then, we read train/validation data

In [4]:
X_trn, y_trn, input_shape = utils.load_data('data/sample/train.txt', 'data/sample/spc8', 'data/sample/negpeaks',
                                            'data/sample/targets', frame_length=0.03, winfunc=None)

and check the shape of inputted data

In [5]:
print('Input shape: ', input_shape)
print('# of training examples:', X_trn.shape[0])
print('# of samples per frame:', X_trn.shape[1])

Input shape:  (240, 1)
# of training examples: 10990
# of samples per frame: 240


In [6]:
X_val, y_val, input_shape = utils.load_data('data/sample/val.txt', 'data/sample/spc8', 'data/sample/negpeaks',
                                            'data/sample/targets', frame_length=0.03, winfunc=None)

In [7]:
print('# of validation examples:', X_val.shape[0])

# of validation examples: 457


In this example, we use 1D version of the InceptionV3 model which is shown in the paper to achieve best results on the test set. The definition of the model is as follows:

In [8]:
# Model definition
model = InceptionV31D(input_shape)
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Then, we can train the model on the train set and evaluate it on the validation set:

In [9]:
history = model.fit(X_trn, y_trn, validation_data=(X_val, y_val), epochs=2, batch_size=128, verbose=1)


Train on 10990 samples, validate on 457 samples
Epoch 1/2
10990/10990 [==============================] - 274s 25ms/step - loss: 0.1354 - accuracy: 0.9495 - val_loss: 0.4859 - val_accuracy: 0.8249
Epoch 2/2
10990/10990 [==============================] - 280s 25ms/step - loss: 0.1002 - accuracy: 0.9599 - val_loss: 0.6024 - val_accuracy: 0.8271


In this very simplified example, the accuracy on the validation set was about 83%. Much better results can be obtained when more training data from more voice talents is used, when tuning of the hyper-parameters (such as the frame size, batch size, learning rate, etc.) is done and also when the model is trained for more epochs. Please see the [paper](paper/Matousek_ICASSP2021_paper.pdf) for more details.

Since the data is unbalanced, the _accuracy_ score could be confusing. In the [paper](paper/Matousek_ICASSP2021_paper.pdf), we use $F1$, _recall_ ($R$), and _precision_ ($P$) scores. For this purpose, we firstly get the prediction of each peak to be GCI or non_GCI 

In [10]:
# Predict to get some other metrics
y_proba = model.predict(X_val, verbose=1)[:, 0]
y_pred = utils.proba2classes(y_proba)

457/457 [==============================] - 4s 9ms/step


and then we use [Scikit-learn](http://scikit-learn.org/stable/) tools to calculate the measures

In [11]:
print('F1 = {:.2%}'.format(skm.f1_score(y_val, y_pred)))
print('R  = {:.2%}'.format(skm.recall_score(y_val, y_pred)))
print('P  = {:.2%}'.format(skm.precision_score(y_val, y_pred)))

F1 = 90.52%
R  = 100.00%
P  = 82.68%


## Predicting with the CNN model

We have tuned the hyper-parameters of the InceptionV3-1D model and trained it on all data (3200 utterances) using GPU (see the [paper](paper/Matousek_ICASSP2021_paper.pdf)). The resulting pre-trained weigths are available [here](prediction/weights.h5): `prediction/weights.h5` and the model's architecture is [here](prediction/architecture.json): `prediction/architecture.json`. They could be used to set up the model

In [12]:
with open('prediction/architecture.json', 'rt') as json_file:
    model = model_from_json(json_file.read())
# Load optimal model's weights from hdf5 file
model.load_weights('prediction/weights.h5')

For prediction, we have to pre-process the input waveform to meet the format used when the model was trained. The following steps have to be done:
1. The input waveform is downsampled to 8 kHz and mastered/normalized
1. The waveform is then lowpass-filtered (with the lowpass filter coefficient stored in `prediction/filtcoef800.npy`).
1. Negative peaks are detected using the lowpass-filtered signal.

The function `preprocess` could be used to do that:

In [13]:
# Read filter coefficients
filtcoef = np.load('prediction/filtcoef800.npy')
# Read input waveform (16 kHz)
samples_src, sf_src = lr.load('prediction/wav/slt_arctic_a0001.wav', sr=None)
# Pre-process the waveform
samples_tgt, peaks, filtsamples = utils.preprocess(samples_src, sf_src, 8000, filtcoef, norm_amp_spc=30000,
                                                   norm_amp_filt=0.9)
print('# neg. peaks:', len(peaks))

# neg. peaks: 457


Next, we extract samples around each negative peak, this time using a 80ms frame and Hamming window, and store the input data representing it as time steps

In [14]:
data_list = utils.frames_from_utt(samples_tgt, 8000, peaks, 0.080, np.hamming)
X = utils.data_as_timesteps(data_list)

Now, we can make the prediction. In our case, it means to assign each negative a probabilistic prediction: whether this peak represents a GCI (prediction > 0.5) or a non-GCI (prediction <= 0.5)

In [15]:
# Predict GCIs => get a probabilistic prediction per a peak
y = model.predict(X).flatten()
print('# predictions (1/0): {} ({}/{})'.format(len(y), len(y[y > 0.5]), len(y[y <= 0.5])))

# predictions (1/0): 457 (391/66)


Since our predictions were made for 8kHz signals, we must convert them to correspond with the source sampling frequency (16 kHz in our case). We also convert the predictions from samples to time marks (in seconds). 

In [16]:
# Convert peak indices according to SOURCE sampling frequency
peaks_src = gu.seconds2samples(gu.samples2seconds(peaks, 8000), sf_src)
# Get samples of the predicted GCIs synced to the nearest peak in the SOURCE signal
pred_samps_src = gu.sync_predictions_to_samp_peak(y, peaks_src, samples_src,
                                                  gu.seconds2samples(0.0015, sf_src),
                                                  gu.seconds2samples(0.0020, sf_src))
pred_times_src = gu.samples2seconds(pred_samps_src, sf_src)

Finally, we store the detected GCIs in the [wavesurfer](http://www.speech.kth.se/wavesurfer) format (see below) using the `Pitchmark` object

In [17]:
# Create pitch-marks from GCIs
pms = gu.create_gci(pred_times_src)
# Write GCIs to a file
pms.write_file('prediction/gci_pred/slt_arctic_a0001.pm')

And this is the example of the detected GCIs:

![GCI detection sample](figs/gci_detection_sample.png "GCI detection sample")

For multiple waveforms, the prediction can be run using the following script (please ignore the deprecation warnings)

In [18]:
%%bash
cd prediction
./detect_gci.py --architecture=architecture.json  \
                --weights=weights.h5              \
                --filt-coef=filtcoef800.npy       \
                --sf-tgt=8000                     \
                --frame-length=0.080              \
                --winfunc=hamming                 \
                --sync-left=0.0015                \
                --sync-right=0.0020               \
                './wav/*.wav'                     \
                './gci_pred'
cd ..

Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
WARNING    From /Users/jmatouse/GIT_repos/ARTIC/2021_ICASSP/.direnv/python-3.7.5/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

WARNING    From /Users/jmatouse/GIT_repos/ARTIC/2021_ICASSP/.direnv/python-3.7.5/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.


WARNING    From /Users/jmatouse/GIT_repos/ARTIC/2021_ICASSP/.direnv/python-3.7.5/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4074: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

2020-11-03 16:20:17.070944:

## Evaluating on CMU data

The trained and tuned CNN model can be evaluated on the [CMU](http://festvox.org/dbs/index.html) test datasets. Again, we used the [Multi-Phase Algorithm](http://www.sciencedirect.com/science/article/pii/S0167639311000094) (MPA) to detect GCIs from the contemporaneous electroglottograph (EGG) signal and used them as the ground truth ones. The ground truth GCIs are available in the [wavesurfer](http://www.speech.kth.se/wavesurfer) format

```
0.234687 0.234687 V
0.242312 0.242312 V
0.250250 0.250250 V
0.258062 0.258062 V
0.265937 0.265937 V
```

The most important is the first column which denotes the position of a GCI in seconds. Other columns can be ignored.

The ground truth GCIs for all utterances and voices can be found in the respective subfolders of the ```data/evaluation``` folder, or here:
* [BDL](data/evaluation/bdl/bdl_gt_gci.tar.gz)
* [SLT](data/evaluation/slt/slt_gt_gci.tar.gz)
* [KED](data/evaluation/ked/ked_gt_gci.tar.gz)

GCIs detected by different methods are stored in the ``data/evaluation/<voice>`` folder where ``<voice>`` is one of the voices we experimented with: ``bdl``, ``slt``, ``ked``.

The name of the compressed file with GCIs is as follow:

``<voice>_<method>_<type>_gci``
* ``<voice>``  ...  a voice (``bdl``, ``slt``, ``ked``)
* ``<method>`` ... GCI detection method (``dypsa``, ``mmf``, ``reaper``, ``sedreams``, ``gefba``, ``psfm``, ``xgboost``, ``inceptionV31D``)
* ``<type>``   ... GCI type (original vs. postprocessed)
  * ``orig`` ... original GCIs as detected by each method
  * ``post`` ... postprocessed GCIs (V/U filtering, syncing with neighboring minimum negative sample)

### Example of GCI detection evaluation
Here is an example of the evaluation of GCI detection in terms of identification rate (IDR), miss rate (MR), false alarm rate (FAR), identification rate (IDA), and accuracy within 0.25 ms range (A25).

In [19]:
%%bash
cd prediction
./eval_gci.py gci_gt gci_pred > eval_results.csv
cd ..

INFO       slt_arctic_a0001    : IDR = 96.78%
INFO       slt_arctic_a0002    : IDR = 93.77%
INFO       slt_arctic_a0003    : IDR = 97.97%
INFO       TOTAL               : IDR = 96.23%


where
* `gci_gt` ... directory with ground truth GCIs
* `gci_pred` ... directory with detected (and postprocessed) GCIs
* `eval_results.csv` ... results for each utterance from `gci_gt` and `gci_pred` directories, and total GCI detection results for all uterances.

We can see that the total _identification rate_ (IDR) for our three testing waveforms was 96.23%.

Any of the `<voice>_<method>_post_gci` and the corresponding ``<voice>_gti_gci`` directories (after decompressing from `data/evaluation/<voice>`) can be used to reproduce the results described in the [paper](paper/matousek_ICASSP2021_paper.pdf).

For instance, the results for the BDL voice and SEDREAMS method can be obtained by calling:

``eval_gci.py bdl_gt_gci bdl_sedreams_post_gci > eval_results_bdl_sedreams.csv``